In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data into DataFrames
train_users = pd.read_csv('./data/train_users_2.csv')
test_users = pd.read_csv('./data/test_users.csv')

In [3]:
train_users.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [4]:
# Replace null
train_users.gender.replace('-unknown-', np.nan, inplace=True)

In [4]:
cat_features = ['gender','age','signup_method','signup_flow','language','affiliate_channel','affiliate_provider', \
                'first_affiliate_tracked','signup_app','first_device_type','first_browser']
target = ['country_destination']

In [5]:
for feature in cat_features:
    print train_users[feature].value_counts()

-unknown-    95688
FEMALE       63041
MALE         54440
OTHER          282
Name: gender, dtype: int64
30      6124
31      6016
29      5963
28      5939
32      5855
27      5738
33      5527
26      5044
34      5029
35      4860
25      4459
36      4083
37      3694
38      3384
24      3220
39      2998
40      2766
41      2538
23      2462
42      2243
45      2149
44      2137
43      2056
46      1875
22      1702
47      1646
48      1469
50      1387
51      1337
49      1331
        ... 
94        12
97        10
15         8
2          7
113        4
4          3
1932       3
1949       3
1931       3
1947       2
1936       2
1          2
1929       2
1928       2
111        2
1924       2
2008       1
112        1
132        1
150        1
1925       1
1995       1
1935       1
1953       1
1952       1
1926       1
1927       1
1942       1
1938       1
1933       1
Name: age, dtype: int64
basic       152897
facebook     60008
google         546
Name: signup_method, dt

In [6]:
train_users.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

In [7]:
train_users.shape

(213451, 16)

In [8]:
train_users.count()

id                         213451
date_account_created       213451
timestamp_first_active     213451
date_first_booking          88908
gender                     213451
age                        125461
signup_method              213451
signup_flow                213451
language                   213451
affiliate_channel          213451
affiliate_provider         213451
first_affiliate_tracked    207386
signup_app                 213451
first_device_type          213451
first_browser              213451
country_destination        213451
dtype: int64

In [9]:
len(train_users[train_users['age'] > 90]), len(train_users[train_users['age'] < 90]), len(train_users[train_users['age'].isnull()])

(2543, 122900, 87990)

In [10]:
train_users = train_users.fillna({'age':0})

In [11]:
len(train_users[train_users['age'] > 90]), len(train_users[train_users['age'] < 90]), len(train_users[train_users['age'].isnull()])

(2543, 210890, 0)

In [12]:
train_users = train_users[train_users['age'] < 90]

In [13]:
train_users.shape

(210890, 16)

In [14]:
train_data = pd.get_dummies(train_users[cat_features])

In [15]:
train_data.head(10).shape

(10, 132)

In [16]:
train_data.isnull().sum()

age                                   0
signup_flow                           0
gender_-unknown-                      0
gender_FEMALE                         0
gender_MALE                           0
gender_OTHER                          0
signup_method_basic                   0
signup_method_facebook                0
signup_method_google                  0
language_ca                           0
language_cs                           0
language_da                           0
language_de                           0
language_el                           0
language_en                           0
language_es                           0
language_fi                           0
language_fr                           0
language_hr                           0
language_hu                           0
language_id                           0
language_is                           0
language_it                           0
language_ja                           0
language_ko                           0


In [17]:
#train_data = train_data.fillna({'age':0})

In [18]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(n_estimators=10)
model1.fit(train_data, train_users[target])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
model1.feature_importances_.shape

(132,)

In [20]:
sorted(zip(train_data.columns.tolist(), model1.feature_importances_.tolist()), key=lambda x: x[1], reverse=True)[:20]

[('age', 0.59311254540744),
 ('signup_flow', 0.05604890828794147),
 ('gender_-unknown-', 0.036488961277635804),
 ('signup_method_facebook', 0.027167371790925033),
 ('signup_method_basic', 0.02181720423555077),
 ('gender_FEMALE', 0.019008759533083334),
 ('gender_MALE', 0.014151136122209673),
 ('first_affiliate_tracked_untracked', 0.013898140114303284),
 ('first_affiliate_tracked_linked', 0.009927072811649226),
 ('first_browser_Chrome', 0.00931768284281363),
 ('first_affiliate_tracked_omg', 0.009063546456920396),
 ('first_browser_-unknown-', 0.00845473708794252),
 ('first_browser_Firefox', 0.008102990071948068),
 ('first_device_type_Mac Desktop', 0.007937410068649915),
 ('language_en', 0.007557978446616041),
 ('affiliate_channel_content', 0.007118901848859452),
 ('first_device_type_Windows Desktop', 0.0070095681505426876),
 ('first_browser_Safari', 0.006136364287327568),
 ('affiliate_channel_sem-brand', 0.006005748501432789),
 ('affiliate_channel_sem-non-brand', 0.0059912025601636726)]

In [21]:
model1.score(train_data.values,train_users[target].values)

0.71695196547963391

In [22]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data.values, train_users[target].values, test_size=0.33, random_state=42)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((141296, 132), (69594, 132), (141296, 1), (69594, 1))

In [24]:
model2 = RandomForestClassifier(n_estimators=10)
model2.fit(X_train, y_train)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
model2.score(X_train, y_train)

0.72844241875212323

In [26]:
model2.score(X_test, y_test)

0.59564042877259538

In [27]:
sorted(zip(train_data.columns.tolist(), model1.feature_importances_.tolist()), key=lambda x: x[1], reverse=True)[:10]

[('age', 0.59311254540744),
 ('signup_flow', 0.05604890828794147),
 ('gender_-unknown-', 0.036488961277635804),
 ('signup_method_facebook', 0.027167371790925033),
 ('signup_method_basic', 0.02181720423555077),
 ('gender_FEMALE', 0.019008759533083334),
 ('gender_MALE', 0.014151136122209673),
 ('first_affiliate_tracked_untracked', 0.013898140114303284),
 ('first_affiliate_tracked_linked', 0.009927072811649226),
 ('first_browser_Chrome', 0.00931768284281363)]

In [28]:
from sklearn.cross_validation import cross_val_score

In [29]:
X_train.shape, y_train.shape

((141296, 132), (141296, 1))

In [30]:
c, r = y_train.shape
y_train = y_train.reshape(c,)

In [31]:
model3 = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(model3, X_train, y_train, cv=5)

In [32]:
scores

array([ 0.59291628,  0.59290188,  0.59348903,  0.59460703,  0.59361506])

In [33]:
from sklearn.linear_model import LogisticRegression
model4 = LogisticRegression()
scores = cross_val_score(model4, X_train, y_train, cv=5)
print scores

[ 0.60986484  0.60755104  0.60651097  0.61028345  0.61046224]


In [44]:
model4.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
model4.coef_.shape, X_train.shape

((12, 131), (141296, 131))

In [56]:
#from sklearn.svm import LinearSVC
#model5 = LinearSVC()
#model5 = cross_val_score(model5, X_train, y_train, cv=5)

In [57]:
#model5

array([ 0.58435355,  0.59311419,  0.5604034 ,  0.58452175,  0.55730162])